# Ants on a stick

## Task description

Imagine there is a stick 1 meter long. We drop n ants on this stick in random places (where n is not known beforehand). Ants travel along a stick at a speed of 1 cm/s. When they meet on their path, they will change direction (bounce off each other). Ants can fall from either side of the stick. How much time must elapse till we are sure all the ants will fall off the said stick?

## Line of reasoning

Single ant placed on one end of the stick facing the other will travel whole length of a stick.

![title](img/1.png)

Two ants placed on opposite ends of the stick, facing each other, will bounce in the middle. Each of them will therefore travel length of the stick (once again).

![title](img/2.png)

Question comes to mind: Is it possible for an ant to travel more than half of the stick without bouncing off?

I believe not. Only way for it to be possible would be for an ant to have no other ants facing them on their path. 

As an example to support this claim, you can imagine following scenario: Two ants are placed on opposite ends of the stick, facing each other. One additional ant is placed in between them. 

![title](img/3.png)

The length of the stick is x + 2y. When the ant 2 and 3 meet ant 1 and 2 will be still in distance x relative to each other. Ant 3 will bounce off to the right being on the way to falling.

![title](img/4.png)

Now ant 1 and 2 will travel distance equal to x/2 each. They will bounce off each other and go on their merry way to certain doom (or at least fall :P ).

Ant 1 traveled: y + x/2 + (y + x/2) = x + 2y

Ant 2 traveled: y + x/2 + (x + 2y - (x + y - x/2)) = y + x/2 + (x/2 + y) = x + 2y

Ant 3 traveled: y + y = 2y

Each of these distances is less than or equal the length of the stick. Moreover, the biggest distance in between bounces is equal to half the stick length.

<font color='blue'>More or less this was the moment when we ended our call. I was torn apart at this moment if the answer was 100s or 200s (if ants can only travel length of a stick or twice that much).</font>

Additional question: Is it possible for the ants to bounce off each other infinitely? 

No. They will fall off eventually for sure. 

Ants also cannot catch up with each other because they have the same speed. So we know for sure that ants will bounce off always in mid point between each other.

When we add more ants, situation looks more complicated... at first.

![title](img/5.png)

It is difficult to visualize the path of the ants over time because you would have to check the position of the ants at every single collision. I did my best in image above.

As we agreed at the beginning, 2 ants, after bouncing from each other, after some time return to their initial positions with the opposite motion vector.

We do not need to track the exact path of a single ant. All we care about is question "Is there an ant on specific location in the certain moment?".

Having that in mind, we can abandon keeping track of specific ants. Instead we check only if there is an ant walking selected path during specific time. Now our visualization would look like this:

![title](img/6.png)

As we can clearly see, longest path the (any) ant would travel (with swapping) would be the length of the stick. 

Out initial observations make it happen (ants returning to their orginal positions over time and maximum possible travel distance without bouncing off is half the stick).

Therefore, the minimum time that must elapse before we can be sure all the ants will fall off the said stick is: 

$$
  \frac{L}{a} = \frac{100cm}{1 \frac{cm}{s}} = 100s
$$

where 'L' is stick length and 'a' is ant speed

Is it an acceptable answer? I hope so, because it would be pretty hard to describe mathematically. <font color='blue'>(as promised I didn't check it on the web)</font>

We can also try to estimate the time using heuristic approach. We can place ants in random spots and simulate ants with their collisions. It wouldn't give us definitive answer, but will guide us toward the right answer.

Following code tries to do just that:

In [1]:
# importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# function for generating dataframe with random ants
def Generate_Random_Ants(number, stick_length):
    df = pd.DataFrame() 
    df['position'] = np.random.uniform(0, stick_length, number)
    df = df.sort_values(by=['position'])
    df['facing_right'] = np.random.choice([False, True], number)
    df.insert(0, "ant_id", range(0, number), True)
    
    return df

In [3]:
# function that runs simulation for ant movement and collisions
def Perform_Simulation(number_of_ants, stick_length):
    # our generated dataframe with ants
    df = Generate_Random_Ants(number_of_ants, stick_length)
    
    # necessary variables
    continue_simulation = True
    total_distance = 0.0

    while continue_simulation:
        # calculating distance to nearest impact
        df['nearest_impact'] = np.where((df['facing_right'] == True) & (df['facing_right'].shift(-1) == False), 
                                          (df['position'].shift(-1) - df['position']) / 2, 
                                          np.nan)

        # calculating minimal distance to nearest impact
        min_nearest_impact = df['nearest_impact'].min()


        # bouncing off the ants
        if np.isnan(min_nearest_impact):
            continue_simulation = False
        else:
            # adding minimal distance to nearest impact to total distance
            total_distance += min_nearest_impact

            # updating ants' positions
            df['position'] = np.where(df['facing_right'] == True, 
                                            np.clip(df['position'] + min_nearest_impact, 0, stick_length), 
                                            np.clip(df['position'] - min_nearest_impact, 0, stick_length))
            # updating ants' vector direction
            df['facing_right'] = np.where(df['nearest_impact'] == min_nearest_impact, 
                                            ~df['facing_right'], 
                                            df['facing_right'])
            df['facing_right'] = np.where(df['nearest_impact'].shift(1) == min_nearest_impact, 
                                            ~df['facing_right'], 
                                            df['facing_right'])

    # calculating distance to fall for each ant
    df['dist_to_fall'] = np.where(df['facing_right'] == True, 
                                            stick_length - df['position'], 
                                            df['position'])
    # adding longest distance to fall to total distance
    total_distance  += df['dist_to_fall'].max()
    
    return total_distance

In [4]:
# getting longest distance travelled by ant
longest_ant_path = Perform_Simulation(100, 100)
longest_ant_path

99.59109893872782

In [5]:
# setting ant speed
ant_speed = 1

In [6]:
# calculating total time 
total_time = longest_ant_path / ant_speed
total_time

99.59109893872782